In [ ]:
from transformers import  AutoTokenizer, AutoModelForMaskedLM, pipeline
from datasets import load_dataset
from itertools import groupby
import json

In [2]:
device = 'cuda:0'
hug_token = ''

In [ ]:
# Initialize the model and pipeline
model_name_bert = "distilroberta-base"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = AutoModelForMaskedLM.from_pretrained(model_name_bert)
fill_mask = pipeline("fill-mask", model=model_bert, tokenizer=tokenizer_bert, device=device)

In [4]:
test_dataset = load_dataset("locuslab/TOFU", "forget05")['train']
retain_test_dataset = load_dataset("locuslab/TOFU", "retain95")['train']

In [7]:
def identify_common_words(sentence, length, threshold=0.9):
    # Split sentence into words and track positions
    words = sentence.split()
    common_words = []

    # Get start and end positions for each word
    word_start_end_positions = []
    start_idx = 0
    for word in words:
        start_idx = sentence.find(word, start_idx)  # Find word's start index in the sentence
        end_idx = start_idx + len(word)             # Calculate end index
        word_start_end_positions.append((word, start_idx, end_idx))
        start_idx = end_idx                         # Update start_idx for the next search

    # Create masked sentences in a batch
    masked_sentences = [sentence[:start] + "<mask>" + sentence[end:] for _, start, end in word_start_end_positions]

    # Get predictions in a batch
    all_predictions = fill_mask(masked_sentences, top_k=1)

    # Identify non-common words based on predictions
    for i, (word, start, end) in enumerate(word_start_end_positions):
        # Check if the original word's probability is below the threshold
        # common
        for pred in all_predictions[i]:
            if start>= length and pred['token_str'].strip().lower() == word.lower() and pred['score'] >= threshold:
                common_words.append((word, start-length, end-length))

    return common_words

In [ ]:
def add_common_words(sample, threshold=0):
    common_words = identify_common_words(sample['question'] +' '+ sample['answer'], len(sample['question']) +1,threshold)
    sample['common_words'] = [json.dumps({"word": word, "start": start, "end": end}) for word, start, end in common_words]
    return sample

# Apply the function to each sample in the dataset and save the modifWied dataset
test_dataset = test_dataset.map(lambda x: add_common_words(x))
test_dataset.save_to_disk("../data/forget05_with_common_words")
